# 接口 从协议到抽象基类
## 2 python喜欢序列

In [2]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
f = Foo()
f[1]

10

In [3]:
10 in f

True

In [5]:
for i in f:print(i)

0
10
20


# 3 使用猴子补丁在运行时实现协议

In [7]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks for suit in self.suits]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, pos):
        return self._cards[pos]

In [9]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[5, 3, 2, 0, 7, 8, 9, 6, 1, 4]

In [10]:
d = FrenchDeck()
shuffle(d)

TypeError: 'FrenchDeck' object does not support item assignment

In [11]:
def set_cards(deck, pos, card):
    deck._cards[pos] = card
    
FrenchDeck.__setitem__ = set_cards
shuffle(d)

In [13]:
d[:4]

[Card(rank='A', suit='diamonds'),
 Card(rank='9', suit='spades'),
 Card(rank='9', suit='diamonds'),
 Card(rank='A', suit='spades')]

## 4 Alex的水禽
## 5 定义抽象基类的子类

In [ ]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks for suit in self.suits]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, pos):
        return self._cards[pos]
    
    def __setitem__(self, pos, card):
        self._cards[pos] = card
    
    def __delitem__(self, pos):
        del self._cards[pos]
        
    def insert(self, pos, value):
        self._cards.insert(pos, value)

## 7 定义并使用一个抽象基类

In [16]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""
        
    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回"""
    
    def loaded(self):
        """如果至少有一个元素，返回True，否则False"""
        return bool(self.inspect())
    
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))
    
class Fake(Tombola):
    
    def pick(self):
        return 13
Fake

__main__.Fake

In [17]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

### 7.2 定义Tombola抽象基类子类

In [20]:
import random

class BingoTombola(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return slef.pick()
    

In [22]:
class LotteryTombola(Tombola):
    
    def __init__(self, iterable):
        self._balls - list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            pos = random.randomrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottertBlower')
        return self._balls.pop(pos)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

### 7.3Tombola 的虚拟子类

In [24]:
@Tombola.register
class TombolaList(list):
    
    def pick(self):
        if self:
            pos = random.randrange(len(self))
            return self.pop(pos)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [25]:
issubclass(TombolaList, Tombola)

True

In [27]:
t = TombolaList(range(100))
isinstance(t, Tombola)

True

In [29]:
TombolaList.__mro__

(__main__.TombolaList, list, object)